In [62]:
library(stargazer)
library(Hmisc)
library(ggplot2)
library(sandwich)
library(lmtest)
library(multcomp)
library(data.table)
#library(gdata)

In [97]:
rm(list=ls())
set.seed(100)

In [98]:
n=100 # sample size taken from each state
timeEffect=5 # the effect on y due to time change
policyEffect=3 # the effect on y due to policy change (in NJ)
njBase = 12 # the mean y in NJ in 1998
paBase = 10 # the mean y in PA in 1998

sd.nj=1 # SD of y in NJ
sd.pa=1 # SD of y in PA

In [99]:
d.nj.98 <- data.table(id=1:n, y=rnorm(n,njBase,sd.nj), yr=98, isNJ=T)
d.nj.99 <- data.table(id=1:n, y=rnorm(n,njBase+timeEffect+policyEffect,sd.nj), yr=99, isNJ=T)

d.pa.98 <- data.table(id=(n+1):(2*n), y=rnorm(n,paBase,sd.pa), yr=98, isNJ=F)
d.pa.99 <- data.table(id=(n+1):(2*n), y=rnorm(n,paBase+timeEffect,sd.pa), yr=99, isNJ=F)

In [100]:
d <- rbind(d.nj.98, d.nj.99, d.pa.98, d.pa.99)

In [101]:
dw <- reshape(d, 
  timevar = "yr",
  idvar = c("id", "isNJ"),
  direction = "wide")

In [102]:
dl <- data.table(d)
dl$time <- dl$yr == 99

In [103]:
dw$ch.y <- dw$y.99 - dw$y.98

In [104]:
head(dw)

,id,isNJ,y.98,y.99,ch.y
1,1.000000,1.000000,11.497808,19.667077,8.169269
2,2.000000,1.000000,12.131531,21.363114,9.231583
3,3.00000,1.00000,11.92108,19.53085,7.60977
4,4.000000,1.000000,12.886785,20.842876,7.956091
5,5.000000,1.000000,12.116971,18.542006,6.425035
6,6.000000,1.000000,12.318630,19.599694,7.281064


In [105]:
m1 <- lm(ch.y ~ isNJ, data = dw)
coeftest(m1, vcov = vcovHC)


t test of coefficients:

            Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  4.90396    0.15894  30.853 < 2.2e-16 ***
isNJTRUE     3.10427    0.21073  14.731 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [106]:
m2 <- lm(y ~ isNJ + time + isNJ*time, data=dl)
coeftest(m2, vcov = vcovHC)


t test of coefficients:

                  Estimate Std. Error t value  Pr(>|t|)    
(Intercept)       10.01279    0.10445  95.860 < 2.2e-16 ***
isNJTRUE           1.99012    0.14640  13.593 < 2.2e-16 ***
timeTRUE           4.90396    0.15070  32.541 < 2.2e-16 ***
isNJTRUE:timeTRUE  3.10427    0.19909  15.592 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
